<a href="https://colab.research.google.com/github/PrajaktaPatil1/benign_malignant/blob/master/benign_malignant(with_LBP_feature).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python scikit-image scikit-learn


In [2]:
import os
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import files
uploaded = files.upload()

Saving archive (4).zip to archive (4).zip


In [14]:
import os
import zipfile

# Path to the zip file
zip_file_path = '/content/archive (4).zip'

# Directory where you want to extract the files
extracted_dir = '/content/extracted_data'

# Ensure the directory exists
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Contents of the extracted directory:")
for file_name in extracted_files:
    print(file_name)

# Destination directory to save the extracted files
save_dir = '/content/saved_files'


Contents of the extracted directory:
archive (4)


# Step 1: Data Preprocessing

In [19]:
import cv2

In [20]:
def extract_lbp_features(image):
    gray = rgb2gray(image)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7) # Normalize
    return hist

def load_data(folder_path):
    X = []
    y = []
    for label, class_name in enumerate(['benign', 'malignant']):
        class_folder = os.path.join(folder_path, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            image = cv2.imread(img_path)
            features = extract_lbp_features(image)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

In [21]:
train_folder = "/content/extracted_data/archive (4)/train"
test_folder = "/content/extracted_data/archive (4)/test"

In [24]:
X_train, y_train = load_data(os.path.join(train_folder))

In [22]:
X_test, y_test = load_data(os.path.join(test_folder))

In [25]:
# Assuming y_train is a numpy array containing the training labels
unique_labels_train = np.unique(y_train)

print("Unique labels in y_train:", unique_labels_train)

Unique labels in y_train: [0 1]


# Step 2: Model Training

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [29]:
# Step 1: Train Logistic Regression classifier
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [30]:
# Step 5: Evaluate model
y_pred_lr = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

Accuracy: 0.5591397849462365


In [31]:
from sklearn.metrics import accuracy_score, classification_report

In [32]:
accuracy_score(y_pred_lr,y_test)
print(classification_report(y_pred_lr,y_test))

              precision    recall  f1-score   support

           0       0.41      0.66      0.51        32
           1       0.74      0.51      0.60        61

    accuracy                           0.56        93
   macro avg       0.57      0.58      0.55        93
weighted avg       0.63      0.56      0.57        93



In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(y_pred_lr,y_test)

array([[21, 11],
       [30, 31]])

Implementing a Random Forest Classifier

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
rfc=RandomForestClassifier()

In [37]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [38]:
y_pred_rfc = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rfc)
print("Accuracy:", accuracy)


Accuracy: 0.7204301075268817


In [39]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [40]:
accuracy_score(y_pred_rfc,y_test)
print(classification_report(y_pred_rfc,y_test))

              precision    recall  f1-score   support

           0       0.67      0.79      0.72        43
           1       0.79      0.66      0.72        50

    accuracy                           0.72        93
   macro avg       0.73      0.73      0.72        93
weighted avg       0.73      0.72      0.72        93



In [41]:
confusion_matrix(y_pred_rfc,y_test)

array([[34,  9],
       [17, 33]])

Implementing a KNN

In [42]:
from sklearn.neighbors import KNeighborsClassifier

In [43]:
knn=KNeighborsClassifier(n_neighbors=7)

In [44]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [45]:
y_pred_knn = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [46]:
accuracy_score(y_pred_knn,y_test)
print(classification_report(y_pred_knn,y_test))

              precision    recall  f1-score   support

           0       0.65      0.72      0.68        46
           1       0.69      0.62      0.65        47

    accuracy                           0.67        93
   macro avg       0.67      0.67      0.67        93
weighted avg       0.67      0.67      0.67        93



In [47]:
confusion_matrix(y_pred_knn,y_test)

array([[33, 13],
       [18, 29]])

Implementing a Decision Tree

In [48]:
from sklearn.tree import DecisionTreeClassifier

In [49]:
dtc=DecisionTreeClassifier()

In [50]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [51]:
y_pred_dtc = dtc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_dtc)
print("Accuracy:", accuracy)

Accuracy: 0.7096774193548387


In [52]:
accuracy_score(y_pred_dtc,y_test)
print(classification_report(y_pred_dtc,y_test))

              precision    recall  f1-score   support

           0       0.65      0.79      0.71        42
           1       0.79      0.65      0.71        51

    accuracy                           0.71        93
   macro avg       0.72      0.72      0.71        93
weighted avg       0.72      0.71      0.71        93



In [53]:
confusion_matrix(y_pred_dtc,y_test)

array([[33,  9],
       [18, 33]])

Implementing a Naive Bayes classifier

In [54]:
from sklearn.naive_bayes import GaussianNB

In [55]:
nb=GaussianNB()

In [56]:
nb.fit(X_train, y_train)

GaussianNB()

In [57]:
y_pred_nb = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_nb)
print("Accuracy:", accuracy)

Accuracy: 0.6236559139784946


In [58]:
accuracy_score(y_pred_nb,y_test)
print(classification_report(y_pred_nb,y_test))

              precision    recall  f1-score   support

           0       0.63      0.67      0.65        48
           1       0.62      0.58      0.60        45

    accuracy                           0.62        93
   macro avg       0.62      0.62      0.62        93
weighted avg       0.62      0.62      0.62        93



In [59]:
confusion_matrix(y_pred_nb,y_test)

array([[32, 16],
       [19, 26]])

Xgboost

In [61]:
import xgboost as xgb

In [62]:
# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

In [63]:
# Train the classifier
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [64]:
# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)

In [65]:
# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred_xgb)
print("Accuracy:", accuracy)

Accuracy: 0.6344086021505376


In [66]:
accuracy_score(y_pred_xgb,y_test)
print(classification_report(y_pred_xgb,y_test))

              precision    recall  f1-score   support

           0       0.53      0.73      0.61        37
           1       0.76      0.57      0.65        56

    accuracy                           0.63        93
   macro avg       0.65      0.65      0.63        93
weighted avg       0.67      0.63      0.64        93

